# Decomposition of Effects

This notebook aims to decompose differences in the GDP share of health spending and health levels across countries into health service wedges (prices), efficiency wedges (TFP) and health risk wedges ($\delta_{10},\delta_{11}$). 

Scenarios are run in general equilibrium and $s$ and $h$ differences are aggregated within Europe. Intermediate results are saved as pickle dataframes and the main table is outputted to tex.  The output is Table 6 in the paper. 


In [1]:
from matplotlib import pyplot as plt
import numpy as np
from scipy import stats 
from importlib import reload
import pandas as pd
from itertools import product

In [2]:
from tools import micro
from tools import macro
from tools import params
from tools import distributions as dist
reload(micro)
reload(macro)
reload(params)
reload(dist)

<module 'tools.distributions' from '/Users/flangot/Dropbox/agghealth/notebooks/tools/distributions.py'>

## Creating Table

In [3]:
countries = ['us','fr','sp','it','dk','de','nl','se']
regions = ['us','eu','$\\Delta$']
stats = ['s','h','g','c']
scenarios = ['baseline','price','efficiency','health risks']
results_s = pd.DataFrame(index=scenarios,columns=countries)
results_h = pd.DataFrame(index=scenarios,columns=countries)
results_g = pd.DataFrame(index=scenarios,columns=countries)
results_c = pd.DataFrame(index=scenarios,columns=countries)
results_g

,us,fr,sp,it,dk,de,nl,se
baseline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
price,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
efficiency,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
health risks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
cols = [stats[:-1],regions]
tups = list(product(*cols))
index = pd.MultiIndex.from_tuples(tups, names=['stats', 'regions'])
table = pd.DataFrame(index=scenarios,columns=index)
table

stats           s                  h                  g              
regions        us   eu $\Delta$   us   eu $\Delta$   us   eu $\Delta$
baseline      NaN  NaN      NaN  NaN  NaN      NaN  NaN  NaN      NaN
price         NaN  NaN      NaN  NaN  NaN      NaN  NaN  NaN      NaN
efficiency    NaN  NaN      NaN  NaN  NaN      NaN  NaN  NaN      NaN
health risks  NaN  NaN      NaN  NaN  NaN      NaN  NaN  NaN      NaN

## Organizing Parameters

These are the parameters reported in the paper and coming out of MSM estimation. 

In [5]:
p_com   = ['beta','psi','sigma','phi']
p_name  = ['d1','d2','p','tfp']
# loading results from estimation
pars = pd.read_pickle('../output_JPE/pars.pkl')
pars

,de,dk,fr,it,nl,se,sp,us
beta,0.970000,0.970000,0.970000,0.970000,0.970000,0.970000,0.970000,0.970000
psi,0.161578,0.161578,0.161578,0.161578,0.161578,0.161578,0.161578,0.161578
sigma,2.105317,2.105317,2.105317,2.105317,2.105317,2.105317,2.105317,2.105317
phi,0.397405,0.397405,0.397405,0.397405,0.397405,0.397405,0.397405,0.397405
d1,-1.282523,-1.602088,-1.099845,-0.831846,-1.390736,-1.394792,0.002353,-0.988958
d2,4.029535,4.273080,3.786290,3.917407,3.992196,4.310991,3.434494,3.511651
p,0.836301,0.888409,0.604059,0.695076,0.657280,0.891689,0.642584,1.000000
tfp,1.011023,1.260592,0.922111,0.613212,0.989482,0.795415,0.795095,1.000000


## Decomposition Function

This function allows to run the GE simulation under various scenarios for each countries. 

In [6]:
def eq(co,iprice=None,itfp=None,irisks=None):
    # estimated parameters 
    p = pars.loc[:,co]
    if iprice!=None or itfp!=None or irisks!=None:
        itax = True
    else :
        itax = False
    if iprice==None:
        iprice = p['p']
    if itfp==None:
        itfp = p['tfp']
    if irisks==None:
        irisks = [p['d1'],p['d2']]
    theta = params.flexpars(sigma=p['sigma'],beta=p['beta'],
                          phi=p['phi'],psi=p['psi'],delta_h1=irisks[0],
                          delta_h2=irisks[1],eta=0.0,tfp=itfp,price=iprice)
    # option for the numerical solution
    ne = 10
    m  = 2.5
    if co == 'us':
        op = params.settings(ne=ne,nk=30,maxk=190.0,curv=0.5,nprocs=40)
    else:
        op = params.settings(ne=ne,nk=30,maxk=150.0,curv=0.5,nprocs=40)
    inc = params.incprocess(country=co)
    inc.tauchen(ne=ne,m=m)
    aux = params.auxpars(country=co)  
    #Decision rules
    csumers = micro.bellman(options=op,flex=theta,aux=aux,inc=inc,rent=5.6e-2)
    csumers.compute_cash()
    csumers.itervalue()
    # distribution
    stats = dist.stationary(dp=csumers,nk=100)
    stats.blowup()
    stats.compute()
    # general equilibrium
    eq = macro.equilibrium(stats=stats,taxes=itax,rent=True)
    eq.solve()
    aggs = eq.aggregates()
    hlth = eq.healthreport()
    s    = iprice*aggs.M/aggs.Y
    coy  = (aggs.C+iprice*aggs.M)/aggs.Y
    h    = hlth.pH
    g    = hlth.gradient[2]
    return s,h,coy,g

## Baseline scenario

In [7]:
for co in countries:
    s,h,coy,g = eq(co)
    print('country = ',co,', s = ',s,', h = ',h,', grad4 = ',g)
    results_s.loc['baseline',co] = s
    results_h.loc['baseline',co] = h
    results_c.loc['baseline',co] = coy
    results_g.loc['baseline',co] = g

solved bellman equation (iter, diff cons, diff medexp)  250 0.015172647806789286 0.040704348040279115
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  879  iterations 
grid for k (nk maxk) 100 190.0
solved bellman equation (iter, diff cons, diff medexp)  159 0.0003945553153510417 0.0007746327012334575
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  265  iterations 
grid for k (nk maxk) 100 190.0
rate =  0.0005 demand =  37.31615558961363 supply =  18.665607810617228  tax =  0.20209548699654836
solved bellman equation (iter, diff cons, diff medexp)  250 0.027999973019442903 13.715623320314327
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  1824  iterations 
grid for k (nk maxk) 100 190.0
rate =  0.025927835051546503 demand =  18.988219712656775 supply =  144.31644508828728  tax =  0.20209548699654836
solved bellman equation (iter, diff cons, diff medexp)  150 0.0006539976939476944 0.0009981286035090076
grid for k (nk, maxk

stationary distribution converged in  1611  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.014125372995203354 demand =  19.462134085672698 supply =  21.416981304759148  tax =  0.17769857709459755
solved bellman equation (iter, diff cons, diff medexp)  57 0.0002908116069844091 0.0009062697021544608
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  324  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.013164197046843615 demand =  20.053146731127484 supply =  20.228233286251527  tax =  0.17769857709459755
solved bellman equation (iter, diff cons, diff medexp)  27 0.0002176434355143364 0.0006175847112235289
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  322  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.013070699679784682 demand =  20.112192029071682 supply =  19.509887340797018  tax =  0.17769857709459755
country =  sp , s =  0.07791319558160063 , h =  0.9386359460624468 , grad4 =  1.0921228293917258
solved bellman equation (iter

solved bellman equation (iter, diff cons, diff medexp)  250 0.046240957662737725 15.896899765526722
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  564  iterations 
grid for k (nk maxk) 100 150.0
solved bellman equation (iter, diff cons, diff medexp)  144 0.0007104858060795749 0.0009088591982511218
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  226  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.0005 demand =  41.5395333612529 supply =  8.628110911713955  tax =  0.2006111265433142
solved bellman equation (iter, diff cons, diff medexp)  221 0.00010481213870683348 0.0009791160396588339
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  2000  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.025927835051546503 demand =  18.927892583258306 supply =  112.82683706455634  tax =  0.2006111265433142
solved bellman equation (iter, diff cons, diff medexp)  144 0.0005604781833925543 0.0009898256514020431
grid for k (nk, maxk)

stationary distribution converged in  274  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.02485429409269423 demand =  21.807539078052514 supply =  21.883808940284247  tax =  0.22404940824137168
solved bellman equation (iter, diff cons, diff medexp)  12 0.00014394906917658545 0.0002334092109457231
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  18  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.024804294092694218 demand =  21.835814769213783 supply =  21.903322551505156  tax =  0.22404940824137168
country =  se , s =  0.09115550530516646 , h =  0.9664956117415071 , grad4 =  1.0199847385537184


## Price scenario

We use the average price in Europe

In [8]:
peu = 0.0
for co in countries:
    if co!='us':
        peu += pars.loc['p',co]/7.0
peu

0.745056794643866

In [9]:
for co in countries:
    s,h,coy,g = eq(co,iprice=peu)
    results_s.loc['price',co] = s
    results_h.loc['price',co] = h
    results_g.loc['price',co] = g
    results_c.loc['price',co] = coy

solved bellman equation (iter, diff cons, diff medexp)  250 0.9821376039398171 9.66349572726609
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  2000  iterations 
grid for k (nk maxk) 100 190.0
solved bellman equation (iter, diff cons, diff medexp)  157 0.0004366424662407553 0.0008221004508470742
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  272  iterations 
grid for k (nk maxk) 100 190.0
rate =  0.0005 demand =  37.31615558961242 supply =  18.502450915342916  tax =  0.20209548699654836
solved bellman equation (iter, diff cons, diff medexp)  147 0.00020781525808022394 0.0007077738656775523
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  1798  iterations 
grid for k (nk maxk) 100 190.0
rate =  0.025927835051546503 demand =  18.98821971265617 supply =  144.60828644640074  tax =  0.20209548699654836
solved bellman equation (iter, diff cons, diff medexp)  173 0.0001868060722056697 0.0009293247012098771
grid for k (nk, maxk)

solved bellman equation (iter, diff cons, diff medexp)  179 0.00010901643595317978 0.0007446214011317398
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  269  iterations 
grid for k (nk maxk) 100 190.0
rate =  0.003555847577592334 demand =  33.84258689496815 supply =  21.67127669406237  tax =  0.15511154485416953
solved bellman equation (iter, diff cons, diff medexp)  166 0.00031243450923135185 0.000996371990535927
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  221  iterations 
grid for k (nk maxk) 100 190.0
rate =  0.009807392898772384 demand =  28.176323218115073 supply =  29.651515077230805  tax =  0.15511154485416953
solved bellman equation (iter, diff cons, diff medexp)  36 0.0003160461302877948 0.0008552751235875178
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  125  iterations 
grid for k (nk maxk) 100 190.0
rate =  0.009131598565240198 demand =  28.712183512468744 supply =  28.715082251358226  tax =  0.155111544

solved bellman equation (iter, diff cons, diff medexp)  102 0.00023932821069383436 0.0007733666026439323
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  386  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.015151807396162084 demand =  22.329521672587976 supply =  15.83725493392878  tax =  0.21239162856786592
solved bellman equation (iter, diff cons, diff medexp)  73 0.00017837791091501742 0.0005839412042227821
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  323  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.016410025369334513 demand =  21.535762782269035 supply =  18.737581622411437  tax =  0.21239162856786592
solved bellman equation (iter, diff cons, diff medexp)  73 0.0004419702218561028 0.0006798767903042346
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  222  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.017115198125048685 demand =  21.110692249062744 supply =  20.038381677353172  tax =  0.21239162

solved bellman equation (iter, diff cons, diff medexp)  214 0.00030687253147565485 0.0009655136328703406
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  2000  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.025927835051546503 demand =  16.767877616256197 supply =  83.92328674659356  tax =  0.18666234601191936
solved bellman equation (iter, diff cons, diff medexp)  157 0.0005074239208999742 0.0009601184344347757
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  302  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.008452634743092562 demand =  27.485123154057455 supply =  9.13647675402251  tax =  0.18666234601191936
solved bellman equation (iter, diff cons, diff medexp)  153 0.00026015082506347653 0.0005607292628226901
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  477  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.017836984730216165 demand =  20.689515933998546 supply =  21.736181328909787  tax =  0.1866623

solved bellman equation (iter, diff cons, diff medexp)  6 0.00012493038697103387 0.0006894778637942522
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  386  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.013497019279482618 demand =  19.84523408254744 supply =  20.131988536135733  tax =  0.10861396586791346
solved bellman equation (iter, diff cons, diff medexp)  161 0.00022240803322493008 0.00036682651815489464
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  269  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.0005 demand =  31.68919992729256 supply =  3.9549109287941215  tax =  0.10909170226820204
solved bellman equation (iter, diff cons, diff medexp)  162 0.00028826146003346764 0.0009523053485249733
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  1494  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.025927835051546503 demand =  14.022024263308985 supply =  45.03000190864495  tax =  0.10909170226820204
sol

...

solved bellman equation (iter, diff cons, diff medexp)  45 0.00017470256466589618 0.0009015053948360219
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  245  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.023709341094576536 demand =  20.973300423683984 supply =  20.223456991182232  tax =  0.09706496079863912
solved bellman equation (iter, diff cons, diff medexp)  36 9.496099657990698e-05 0.0005383213159504407
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  98  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.02399315470124697 demand =  20.83530854267869 supply =  20.299637042519798  tax =  0.09706496079863912
solved bellman equation (iter, diff cons, diff medexp)  30 8.438698612467377e-05 0.000562700611919098
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  189  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.02408374908205334 demand =  20.79157416555551 supply =  20.927708482236046  tax =  0.097064960798639

solved bellman equation (iter, diff cons, diff medexp)  250 0.0023523735720001326 0.024739803902690483
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  294  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.01691572760471053 demand =  23.98174735278507 supply =  21.889590189761208  tax =  0.12192341016506979
solved bellman equation (iter, diff cons, diff medexp)  48 0.00027503164594344653 0.0009648169428935205
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  161  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.01786143266317949 demand =  23.353422791147104 supply =  23.206478587085588  tax =  0.12192341016506979
solved bellman equation (iter, diff cons, diff medexp)  18 0.00022423224615986115 0.0007654621247219495
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  106  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.017931684770586532 demand =  23.307818217255747 supply =  23.610890174095864  tax =  0.12192341016

solved bellman equation (iter, diff cons, diff medexp)  73 0.0007619126094908779 0.0009869012789920362
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  825  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.019218879932692355 demand =  24.09087991708778 supply =  30.3367931419898  tax =  0.21446250396327374
solved bellman equation (iter, diff cons, diff medexp)  53 0.0001396778335673865 0.0006784956008516474
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  503  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.018678289633674544 demand =  24.450719138547417 supply =  26.381177366122003  tax =  0.21446250396327374
solved bellman equation (iter, diff cons, diff medexp)  35 8.692325422465785e-05 0.000616423812598299
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  334  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.01861113516615194 demand =  24.496022648706685 supply =  24.886754771857724  tax =  0.214462503963273

solved bellman equation (iter, diff cons, diff medexp)  160 0.0006191209277255361 0.0008292563969547473
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  2000  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.025927835051546503 demand =  20.249749570380168 supply =  77.67287636307337  tax =  0.15041080306367838
solved bellman equation (iter, diff cons, diff medexp)  159 0.0005765340943622022 0.0009687857425504376
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  367  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.01099042435074527 demand =  30.672377861385076 supply =  8.722582537938447  tax =  0.15041080306367838
solved bellman equation (iter, diff cons, diff medexp)  151 0.0002989155420412004 0.0009784175435783027
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  1237  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.02005483457801002 demand =  23.550942024728066 supply =  37.096673549053875  tax =  0.150410803

solved bellman equation (iter, diff cons, diff medexp)  105 0.0005224795392777537 0.0009857292027213305
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  645  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.020761248117594385 demand =  24.32346370723818 supply =  10.013111694679592  tax =  0.1171544104636292
solved bellman equation (iter, diff cons, diff medexp)  81 0.00035643869324175625 0.0009465518104292414
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  738  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.023655300252903272 demand =  22.501456374650495 supply =  17.427187823699697  tax =  0.1171544104636292
solved bellman equation (iter, diff cons, diff medexp)  51 0.00023985844609430984 0.0006389835827818047
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  874  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.02436225791965762 demand =  22.08826579506783 supply =  22.174479016660953  tax =  0.117154410463

In [10]:
results_s

,us,fr,sp,it,dk,de,nl,se
baseline,0.143196,0.101172,0.077913,0.087148,0.090218,0.098887,0.089898,0.091156
price,0.1105,0.127149,0.088294,0.088524,0.072777,0.088705,0.108003,0.073209
efficiency,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
health risks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
results_h

,us,fr,sp,it,dk,de,nl,se
baseline,0.895806,0.958886,0.938636,0.925139,0.971543,0.959213,0.964648,0.966496
price,0.926674,0.951131,0.926073,0.937502,0.979019,0.965843,0.961673,0.975365
efficiency,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
health risks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
results_c

,us,fr,sp,it,dk,de,nl,se
baseline,0.683212,0.719008,0.745412,0.67045,0.732792,0.716013,0.715167,0.674635
price,0.678104,0.7575,0.787545,0.705712,0.77532,0.746123,0.780232,0.711618
efficiency,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
health risks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
results_g

,us,fr,sp,it,dk,de,nl,se
baseline,1.270615,1.069485,1.092123,1.089773,1.027003,1.066153,1.059432,1.019985
price,1.178924,1.087418,1.158895,1.088713,1.01747,1.054674,1.071869,1.015926
efficiency,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
health risks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## TFP scenario

We use the average $A$ (TFP) in Europe. 

In [14]:
Aeu = 0.0
for co in countries:
    if co!='us':
        Aeu += pars.loc['tfp',co]/7.0
Aeu

0.9124184429499073

In [15]:
for co in countries:
    s,h,coy,g = eq(co,itfp=Aeu)
    results_s.loc['efficiency',co] = s
    results_h.loc['efficiency',co] = h
    results_g.loc['efficiency',co] = g
    results_c.loc['efficiency',co] = coy

solved bellman equation (iter, diff cons, diff medexp)  250 0.015172647806789286 0.040704348040279115
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  879  iterations 
grid for k (nk maxk) 100 190.0
solved bellman equation (iter, diff cons, diff medexp)  158 0.0003379419046698473 0.0009310332268839261
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  301  iterations 
grid for k (nk maxk) 100 190.0
rate =  0.0005 demand =  32.15941275720805 supply =  16.03770568494457  tax =  0.20209548699654836
solved bellman equation (iter, diff cons, diff medexp)  133 0.0002756945618660467 0.0007522625836635655
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  1904  iterations 
grid for k (nk maxk) 100 190.0
rate =  0.025927835051546503 demand =  16.364225778763018 supply =  131.31945651858456  tax =  0.20209548699654836
solved bellman equation (iter, diff cons, diff medexp)  178 0.00017234022347611955 0.0009558849850022089
grid for k (nk, 

solved bellman equation (iter, diff cons, diff medexp)  250 0.06605355587836192 12.083495706832979
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  209  iterations 
grid for k (nk maxk) 100 190.0
rate =  0.0005 demand =  32.159412757202325 supply =  15.485703755738312  tax =  0.22037678864919874
solved bellman equation (iter, diff cons, diff medexp)  132 0.0002779348117845082 0.0008415545515880751
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  1905  iterations 
grid for k (nk maxk) 100 190.0
rate =  0.025927835051546503 demand =  16.364225778760133 supply =  129.09487016311562  tax =  0.22037678864919874
solved bellman equation (iter, diff cons, diff medexp)  155 0.0005944781287334777 0.0009605901076774614
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  291  iterations 
grid for k (nk maxk) 100 190.0
rate =  0.003776368306734058 demand =  28.96738214357233 supply =  17.940219694797996  tax =  0.22037678864919874
solved b

solved bellman equation (iter, diff cons, diff medexp)  51 0.0001698283126709832 0.0007371054908063002
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  205  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.0164644766491144 demand =  21.139656953138726 supply =  20.103162735106807  tax =  0.15037077940735416
solved bellman equation (iter, diff cons, diff medexp)  21 0.0003466616335012773 0.0008301850164862401
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  149  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.016778996846798063 demand =  20.952066731479665 supply =  20.462042968634144  tax =  0.15037077940735416
solved bellman equation (iter, diff cons, diff medexp)  17 0.0003863747529999628 0.0008985912762611292
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  125  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.016861228931218 demand =  20.90346550100944 supply =  20.732522464097563  tax =  0.1503707794073541

solved bellman equation (iter, diff cons, diff medexp)  85 0.00029202229619818 0.0007480498860914508
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  863  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.014173203982701919 demand =  24.21101445199391 supply =  23.311244211535744  tax =  0.17769857709459755
solved bellman equation (iter, diff cons, diff medexp)  37 0.0002830253334362709 0.0007717186216176231
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  173  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.014329585257133171 demand =  24.094835211258918 supply =  23.458615237592767  tax =  0.17769857709459755
solved bellman equation (iter, diff cons, diff medexp)  250 0.0005506112440483157 0.0037498018097537056
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  368  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.014686153350346813 demand =  23.83329714710204 supply =  24.91751436929772  tax =  0.17769857709459

solved bellman equation (iter, diff cons, diff medexp)  149 0.0004834873444519161 0.0009727921456494926
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  1653  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.025927835051546503 demand =  17.46921749651551 supply =  64.94040365785521  tax =  0.08760136225515298
solved bellman equation (iter, diff cons, diff medexp)  182 0.0002592435631907364 0.00042980034434947356
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  529  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.011229426344527312 demand =  26.580261510977415 supply =  12.89168298596066  tax =  0.08760136225515298
solved bellman equation (iter, diff cons, diff medexp)  125 0.00035176165591632724 0.000753251895545759
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  1399  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.016667233557195342 demand =  22.460732310572627 supply =  41.46648785326386  tax =  0.08760136

...

solved bellman equation (iter, diff cons, diff medexp)  13 0.00011596645000899741 0.0008737244021972401
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  19  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.023096829478931095 demand =  12.834782229994934 supply =  13.183849783840945  tax =  0.20366166513151698
solved bellman equation (iter, diff cons, diff medexp)  250 0.046240957662737725 15.896899765526722
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  564  iterations 
grid for k (nk maxk) 100 150.0
solved bellman equation (iter, diff cons, diff medexp)  140 0.0005561911476981152 0.0009791724037135907
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  232  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.0005 demand =  35.269740707360086 supply =  7.3239152114696635  tax =  0.2006111265433142
solved bellman equation (iter, diff cons, diff medexp)  144 0.00036161716556470935 0.0009082873572694439
grid for k (nk, max

stationary distribution converged in  64  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.016936553976322023 demand =  20.350049862906467 supply =  20.417476734647387  tax =  0.16042688621241907
solved bellman equation (iter, diff cons, diff medexp)  116 0.0004864462158078453 0.0009337117683578811
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  221  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.011836534414474475 demand =  23.673584512540454 supply =  12.716760621796642  tax =  0.16042688621241907
solved bellman equation (iter, diff cons, diff medexp)  104 0.00041242710348043943 0.0009337577669512598
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  323  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.016886553976322015 demand =  20.378876619848945 supply =  19.857278527857645  tax =  0.16042688621241907
solved bellman equation (iter, diff cons, diff medexp)  133 0.000513683577680446 0.0008848003849131203
grid for k (nk, maxk) 

stationary distribution converged in  223  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.0005 demand =  38.86299108477831 supply =  3.5356984517866255  tax =  0.14638442818883843
solved bellman equation (iter, diff cons, diff medexp)  171 0.0002385123960140234 0.0005929512749069943
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  2000  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.025927835051546503 demand =  17.717107894180703 supply =  64.95732802803715  tax =  0.14638442818883843
solved bellman equation (iter, diff cons, diff medexp)  160 0.0005499610036121538 0.0009979009586800203
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  393  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.011379540145844184 demand =  26.50929172023754 supply =  8.033972482409048  tax =  0.14638442818883843
solved bellman equation (iter, diff cons, diff medexp)  143 0.00028205474873832515 0.0009045832677259114
grid for k (nk, maxk)  30 150.0
stati

solved bellman equation (iter, diff cons, diff medexp)  192 0.00014896490251992134 0.00030394981045134273
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  402  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.011623119377169675 demand =  26.307911709178207 supply =  8.177515691186088  tax =  0.15958207226223034
solved bellman equation (iter, diff cons, diff medexp)  250 0.00016217796781370364 0.002528029125208775
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  1876  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.02000262229685449 demand =  20.63442496504295 supply =  41.67904394706488  tax =  0.15958207226223034
solved bellman equation (iter, diff cons, diff medexp)  110 0.0003989130898744975 0.0009412640583725818
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  595  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.015501195910586508 demand =  23.40586864714401 supply =  13.2440092119184  tax =  0.15958207226

solved bellman equation (iter, diff cons, diff medexp)  6 0.0005354191014670384 0.0008262337173903234
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  142  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.02471375705754148 demand =  28.234067599181586 supply =  28.45316638956573  tax =  0.10766898995777009
solved bellman equation (iter, diff cons, diff medexp)  11 0.00015900374419830499 0.0009949946545795285
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  275  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.02466375705754147 demand =  28.270747985066148 supply =  27.449372612325092  tax =  0.10766898995777009
solved bellman equation (iter, diff cons, diff medexp)  121 0.0006303798700653118 0.0008665867623349754
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  140  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.0005 demand =  60.82296403541154 supply =  4.025003172424234  tax =  0.11020487110755342
solved bel

## Health Risks Scenarios

We use the average $(\delta_{10},\delta_{11})$ in Europe. 

In [16]:
d1eu = 0.0
d2eu = 0.0
for co in countries:
    if co!='us':
        d1eu += pars.loc['d1',co]/7.0
        d2eu += pars.loc['d2',co]/7.0      
d1eu,d2eu

(-1.0856394508051577, 3.9634275743241156)

In [17]:
for co in countries:
    s,h,coy,g = eq(co,irisks=[d1eu,d2eu])
    results_s.loc['health risks',co] = s
    results_h.loc['health risks',co] = h
    results_g.loc['health risks',co] = g
    results_c.loc['health risks',co] = coy    

solved bellman equation (iter, diff cons, diff medexp)  250 1.310025512504879 9.603322108350188
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  837  iterations 
grid for k (nk maxk) 100 190.0
solved bellman equation (iter, diff cons, diff medexp)  158 0.00041995886197909726 0.000687658946077363
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  264  iterations 
grid for k (nk maxk) 100 190.0
rate =  0.0005 demand =  37.316155589614006 supply =  18.369096164688564  tax =  0.20209548699654836
solved bellman equation (iter, diff cons, diff medexp)  140 0.0002988260943781462 0.0007058900168424032
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  2000  iterations 
grid for k (nk maxk) 100 190.0
rate =  0.025927835051546503 demand =  18.988219712657628 supply =  157.3429594808948  tax =  0.20209548699654836
solved bellman equation (iter, diff cons, diff medexp)  250 0.257734574079419 1.889264075154248
grid for k (nk, maxk)  30 190.

stationary distribution converged in  74  iterations 
grid for k (nk maxk) 100 190.0
rate =  0.009163797919294154 demand =  28.68627898851043 supply =  28.696570290319862  tax =  0.13613636409918162
solved bellman equation (iter, diff cons, diff medexp)  250 0.0007374595171855125 0.014272079648149472
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  1284  iterations 
grid for k (nk maxk) 100 150.0
solved bellman equation (iter, diff cons, diff medexp)  250 0.06287019002104888 1.216729146961585
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  251  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.0005 demand =  36.636566421577534 supply =  5.640342190793818  tax =  0.21239162856786592
solved bellman equation (iter, diff cons, diff medexp)  169 0.00019854579898126445 0.0009423741775052008
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  2000  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.025927835051546503 demand =  

stationary distribution converged in  291  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.0076566783285073135 demand =  28.224976716093515 supply =  8.766564662806832  tax =  0.13156305317097952
solved bellman equation (iter, diff cons, diff medexp)  170 0.0007044523690860061 0.000704841036366588
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  485  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.01775346218118031 demand =  20.737553085464608 supply =  23.04241105493945  tax =  0.13156305317097952
solved bellman equation (iter, diff cons, diff medexp)  92 0.00017305880145590535 0.0005880688493345998
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  273  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.016684153538473624 demand =  21.368886631193128 supply =  20.07259306325666  tax =  0.13156305317097952
solved bellman equation (iter, diff cons, diff medexp)  26 0.0002278261547981053 0.0008457900367915272
grid for k (nk, maxk)  30 

solved bellman equation (iter, diff cons, diff medexp)  37 0.00012252753143071615 0.0006076477846939099
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  200  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.017123487703719867 demand =  21.105776445075218 supply =  21.631625195251942  tax =  0.12402606242762287
solved bellman equation (iter, diff cons, diff medexp)  9 8.757919972479655e-05 0.0007995986472693062
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  101  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.017057054509162232 demand =  21.145224225322764 supply =  21.62914864781621  tax =  0.12402606242762287
solved bellman equation (iter, diff cons, diff medexp)  135 0.0005167740833149637 0.0009055734456060449
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  180  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.0005 demand =  36.63656642153477 supply =  6.214346243554436  tax =  0.12452296818391459
solved b

solved bellman equation (iter, diff cons, diff medexp)  250 0.0014935981473992443 14.043685555521376
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  103  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.01393258387302897 demand =  19.578387668995102 supply =  19.63096532683066  tax =  0.17769857709459755
solved bellman equation (iter, diff cons, diff medexp)  250 0.0026844813626077935 14.051009734228952
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  82  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.013864003957198298 demand =  19.620014506703725 supply =  19.62387790120966  tax =  0.17769857709459755
solved bellman equation (iter, diff cons, diff medexp)  250 0.0031794782250260667 14.035350578176546
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  99  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.013814003957198291 demand =  19.650454451896575 supply =  19.613173427372736  tax =  0.17769857709459755
sol

...

solved bellman equation (iter, diff cons, diff medexp)  156 0.0005074158727333966 0.0009578866487878201
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  1459  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.025927835051546503 demand =  19.933034006780986 supply =  37.17264970789982  tax =  0.1306771389275262
solved bellman equation (iter, diff cons, diff medexp)  126 0.0005621017704457998 0.000997032498620598
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  360  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.01784728598082882 demand =  24.196890706750946 supply =  9.22702169077121  tax =  0.1306771389275262
solved bellman equation (iter, diff cons, diff medexp)  92 0.0007563307134113018 0.0009739113459197313
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  357  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.02160284935690561 demand =  22.04609469300201 supply =  14.720392965237902  tax =  0.1306771389275262

stationary distribution converged in  2000  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.025927835051546503 demand =  18.92789258325879 supply =  120.7003916491156  tax =  0.13405248426101316
solved bellman equation (iter, diff cons, diff medexp)  141 0.0005978188397479656 0.0009947636728959708
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  262  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.006612041348956243 demand =  33.09674914253177 supply =  11.44784742202121  tax =  0.13405248426101316
solved bellman equation (iter, diff cons, diff medexp)  134 0.0004948394266763856 0.0009522730829729653
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  291  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.01694957863375886 demand =  23.958784021573962 supply =  21.73401706041837  tax =  0.13405248426101316
solved bellman equation (iter, diff cons, diff medexp)  79 0.0001458945965975289 0.00035535288328247816
grid for k (nk, maxk)  30 

stationary distribution converged in  406  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.0120863953279268 demand =  29.638512204837436 supply =  8.808832548790523  tax =  0.21446250396327374
solved bellman equation (iter, diff cons, diff medexp)  152 0.0002335214699265009 0.0008581712775175276
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  1944  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.0205269255372737 demand =  23.25455822482261 supply =  48.00865511815138  tax =  0.21446250396327374
solved bellman equation (iter, diff cons, diff medexp)  140 0.00027888712602486265 0.0008838054069464363
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  578  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.01594332763360015 demand =  26.411102736289582 supply =  14.444390175027767  tax =  0.21446250396327374
solved bellman equation (iter, diff cons, diff medexp)  111 0.0001780602983023627 0.0008923725308171981
grid for k (nk, maxk)  30 1

solved bellman equation (iter, diff cons, diff medexp)  85 8.10117071190497e-05 0.0009037922117265396
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  309  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.018736773949021582 demand =  24.411374232569706 supply =  27.157895365688866  tax =  0.12389247622574447
solved bellman equation (iter, diff cons, diff medexp)  28 0.00018727843255161858 0.0006382098387405222
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  430  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.018517723934046035 demand =  24.559264073562073 supply =  21.53611656224001  tax =  0.12389247622574447
solved bellman equation (iter, diff cons, diff medexp)  22 6.935801235119499e-05 0.0008484667430224135
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  156  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.018632500110710253 demand =  24.4815948852235 supply =  21.43921830950931  tax =  0.12389247622574

solved bellman equation (iter, diff cons, diff medexp)  27 0.00019795859258753268 0.0009393262030918947
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  380  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.025028479731254324 demand =  21.709473490395315 supply =  21.564460365022967  tax =  0.1706206869766835
solved bellman equation (iter, diff cons, diff medexp)  17 0.00022861084345660032 0.0009645531971944621
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  121  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.025078479731254336 demand =  21.681449096233504 supply =  21.740203833531417  tax =  0.1706206869766835
solved bellman equation (iter, diff cons, diff medexp)  250 0.4566145053701631 3.146567354904425
grid for k (nk, maxk)  30 150.0
stationary distribution converged in  145  iterations 
grid for k (nk maxk) 100 150.0
rate =  0.0005 demand =  47.150197170954705 supply =  3.224041984552531  tax =  0.1670217344351002
solved bellman

In [18]:
results_s

,us,fr,sp,it,dk,de,nl,se
baseline,0.143196,0.101172,0.077913,0.087148,0.090218,0.098887,0.089898,0.091156
price,0.1105,0.127149,0.088294,0.088524,0.072777,0.088705,0.108003,0.073209
efficiency,0.157002,0.10412,0.071436,0.050541,0.14995,0.116159,0.101859,0.071361
health risks,0.09729,0.084541,0.119587,0.096873,0.096098,0.098668,0.087725,0.105763


# Table of results: all scenarios re-computed with the same grid
The grids are differents from the one used in the estimation procedure in order to improve the precision

In [19]:
eu_countries = ['fr','sp','it','dk','de','nl','se']

In [20]:
for scn in scenarios:
    table.loc[scn,('s','us')] = results_s.loc[scn,'us']
    table.loc[scn,('s','eu')] = np.mean(results_s.loc[scn,eu_countries])
    table.loc[scn,('s','$\\Delta$')] = table.loc[scn,('s','us')]-table.loc[scn,('s','eu')]
    table.loc[scn,('h','us')] = results_h.loc[scn,'us']
    table.loc[scn,('h','eu')] = np.mean(results_h.loc[scn,eu_countries])
    table.loc[scn,('h','$\\Delta$')] = table.loc[scn,('h','us')]-table.loc[scn,('h','eu')]
    table.loc[scn,('g','us')] = results_g.loc[scn,'us']
    table.loc[scn,('g','eu')] = np.mean(results_g.loc[scn,eu_countries])
    table.loc[scn,('g','$\\Delta$')] = table.loc[scn,('g','us')]-table.loc[scn,('g','eu')]   

In [21]:
table.round(3)

stats                s                             h                      \
regions             us        eu  $\Delta$        us        eu  $\Delta$   
baseline      0.143196  0.090913  0.052283  0.895806  0.954937 -0.059131   
price           0.1105   0.09238   0.01812  0.926674  0.956658 -0.029984   
efficiency    0.157002  0.095061  0.061941  0.886245  0.957789 -0.071544   
health risks   0.09729  0.098465 -0.001175  0.937464  0.947039 -0.009575   

stats                g                      
regions             us        eu  $\Delta$  
baseline      1.270615  1.060565  0.210051  
price         1.178924  1.070709  0.108215  
efficiency    1.301352  1.057517  0.243834  
health risks  1.161143  1.084043  0.077101

# Table of results where the benchmark and the price scenarios are not with the same grids than the others scenarios

In [ ]:
# Results for benchmark and price scenarios
# s
sb = [0.140133, 0.10057,  0.077059, 0.086972, 0.087288,  0.09904,   0.0889796, 0.0903872]
sp = [0.110341, 0.126691, 0.087393, 0.10693,  0.0730481, 0.0889361, 0.106651,  0.0733584]
# h
hb = [0.896541 , 0.9597 , 0.937642 , 0.925261 , 0.973665 , 0.95948 , 0.965963 , 0.967705]
hp = [0.925708 , 0.950858 , 0.929576 , 0.90776 , 0.978669 , 0.965688 , 0.962528 , 0.975766]
# c
cb = [0.679479 , 0.717629 , 0.737936 , 0.669748 , 0.730785 , 0.716155 , 0.713161 , 0.673503]
cp = [0.677385 , 0.755819 , 0.778857 , 0.738478 , 0.775511 , 0.746577 , 0.775522 , 0.712928]
# g
gb = [1.27547 , 1.06596 , 1.09368 , 1.08953 , 1.02656 , 1.06441 , 1.05499 , 1.01917]
gp = [1.1825 , 1.08826 , 1.13641 , 1.1457 , 1.01842 , 1.05508 , 1.069 , 1.01558]

In [ ]:
results_s.loc['baseline',:] = sb
results_s.loc['price',:]    = sp

results_h.loc['baseline',:] = hb
results_h.loc['price',:]    = hp

results_c.loc['baseline',:] = cb
results_c.loc['price',:]    = cp

results_g.loc['baseline',:] = gb
results_g.loc['price',:]    = gp

In [ ]:
results_s

In [ ]:
results_c

In [ ]:
results_g

## Saving results

In [20]:
results_s.to_pickle('../output_JPE/decomp_share.pkl')
results_h.to_pickle('../output_JPE/decomp_hlth.pkl')
results_c.to_pickle('../output_JPE/decomp_csy.pkl')
results_g.to_pickle('../output_JPE/decomp_gradient.pkl')

## Loading (if not re-running)

In [19]:
results_s = pd.read_pickle('../output_JPE/decomp_share.pkl')
results_h = pd.read_pickle('../output_JPE/decomp_hlth.pkl')
results_c = pd.read_pickle('../output_JPE/decomp_csy.pkl')
results_g = pd.read_pickle('../output_JPE/decomp_gradient.pkl')

## Fill in Results Table

In [11]:
eu_countries = ['fr','sp','it','dk','de','nl','se']

In [22]:
for scn in scenarios:
    table.loc[scn,('s','us')] = results_s.loc[scn,'us']
    table.loc[scn,('s','eu')] = np.mean(results_s.loc[scn,eu_countries])
    table.loc[scn,('s','$\\Delta$')] = table.loc[scn,('s','us')]-table.loc[scn,('s','eu')]
    table.loc[scn,('h','us')] = results_h.loc[scn,'us']
    table.loc[scn,('h','eu')] = np.mean(results_h.loc[scn,eu_countries])
    table.loc[scn,('h','$\\Delta$')] = table.loc[scn,('h','us')]-table.loc[scn,('h','eu')]
    table.loc[scn,('g','us')] = results_g.loc[scn,'us']
    table.loc[scn,('g','eu')] = np.mean(results_g.loc[scn,eu_countries])
    table.loc[scn,('g','$\\Delta$')] = table.loc[scn,('g','us')]-table.loc[scn,('g','eu')]   

In [23]:
table.round(3)

stats                 s                                h            \
regions              us         eu    $\Delta$        us        eu   
baseline       0.140133  0.0900423   0.0500907  0.896541  0.955631   
price          0.110341  0.0947154   0.0156256  0.925708  0.952978   
efficiency      0.15684  0.0949022   0.0619381  0.887038   0.95802   
health risks  0.0970709  0.0986646 -0.00159371  0.937639  0.946186   

stats                           g                      
regions         $\Delta$       us       eu   $\Delta$  
baseline      -0.0590899  1.27547  1.05919   0.216284  
price         -0.0272699   1.1825  1.07549   0.107007  
efficiency    -0.0709818  1.29931  1.05606    0.24325  
health risks -0.00854663  1.16033   1.0844  0.0759311

In [24]:
table.round(3).to_latex('../output_JPE/decomposition.tex')